# LightGraphs Multiplex Extension

## Useful Links

### LightGraphs & related
- [Extend Lightgraph](https://juliagraphs.org/LightGraphs.jl/latest/developing/#Developing-Alternate-Graph-Types-1)

- [SimpleWeightedGraphs - GitHub](https://github.com/JuliaGraphs/SimpleWeightedGraphs.jl)

- [MultiGraphs - GitHub](https://github.com/QuantumBFS/Multigraphs.jl)

- [LightGraphs - GitHub](https://github.com/JuliaGraphs/LightGraphs.jl/tree/master/src)

### Programming in Julia

- [Julia OOP (non exhaustive)_1 ](https://github.com/ninjaaron/oo-and-polymorphism-in-julia/tree/5324b09a08bd27419006868c68c5b0fa6084c8f1)

- [Julia OOP (non exhaustive)_2](http://www.stochasticlifestyle.com/type-dispatch-design-post-object-oriented-programming-julia/)

- [Julia - Collections](https://docs.julialang.org/en/v1/base/collections/)


## Implementation

In [1]:
using LightGraphs
using SimpleWeightedGraphs
import Base

In [2]:
mutable struct MultiplexGraph{T <: Int} <: AbstractGraph{T} #{T <: Integer}
    #n::Int64
    #src = Vector{T}
    follower_graph::SimpleWeightedDiGraph
    retweet_graph::SimpleWeightedDiGraph
    favorite_graph::SimpleWeightedDiGraph
    
    function MultiplexGraph(n::Int) # to be added the src/dst/weights constructor
        follower_graph = SimpleWeightedDiGraph{Int,Int}(n)
        retweet_graph = SimpleWeightedDiGraph{Int,Int}(n)
        favorite_graph = SimpleWeightedDiGraph{Int,Int}(n)
        return new{Int}(follower_graph,retweet_graph,favorite_graph) #(n,
    end
    
end

In [3]:
multiplex = MultiplexGraph(3)

MultiplexGraph{Int64}({3, 0} directed simple Int64 graph with Int64 weights, {3, 0} directed simple Int64 graph with Int64 weights, {3, 0} directed simple Int64 graph with Int64 weights)

### `has_edge`

In [4]:
function has_edge(mp::MultiplexGraph, s::Int, d::Int, which::Symbol)
    return SimpleWeightedGraphs.has_edge(getfield(mp, which) , s , d )
end

has_edge(multiplex, 1,3,:follower_graph)

false

### `get_weight`

In [5]:
function get_weight(mp::MultiplexGraph, s::Int, d::Int, which::Symbol)
    return SimpleWeightedGraphs.weights(getfield(mp, which))[s,d]
end

get_weight(multiplex, 1,2,:follower_graph)

0

### `add_edge!`

In [6]:
function add_edge!(mp::MultiplexGraph, s::Int, d::Int, which::Symbol , weight::Int = 1 )
    current_weight = get_weight(mp,s,d,which)
    if which == :follower_graph && !(weight in [1,-1])
        return error("Error: follower graph weights must be 1.")
    elseif which == :follower_graph && current_weight == 1 && weight == 1
        return error("Error: edge already esisting.")
    else
        if current_weight+weight != 0
            SimpleWeightedGraphs.add_edge!(getfield(mp,which) ,s,d ,current_weight+weight)
        else  
            SimpleWeightedGraphs.rem_edge!(getfield(mp,which) ,s,d)
        end
    end
end

add_edge! (generic function with 2 methods)

### `edges` implementation

In [7]:
function edges(mp::MultiplexGraph, which::Symbol)
    return [edge for edge in SimpleWeightedGraphs.edges(getfield(mp,which)) if edge.weight != 0  ]
end

edges (generic function with 1 method)

In [8]:
add_edge!(multiplex, 1, 2, :follower_graph)
add_edge!(multiplex, 3, 1, :follower_graph)
add_edge!(multiplex, 2, 3, :retweet_graph, 3)
add_edge!(multiplex, 1, 3, :favorite_graph, 2)
println(collect(edges(multiplex, :follower_graph)))
println(collect(edges(multiplex, :retweet_graph)))
println(collect(edges(multiplex, :favorite_graph)))
println(add_edge!(multiplex, 1, 2, :follower_graph, 1))

SimpleWeightedEdge{Int64,Int64}[Edge 1 => 2 with weight 1, Edge 3 => 1 with weight 1]
SimpleWeightedEdge{Int64,Int64}[Edge 2 => 3 with weight 3]
SimpleWeightedEdge{Int64,Int64}[Edge 1 => 3 with weight 2]


LoadError: Error: edge already esisting.

### `eltype` implementation

In [9]:
eltype(mp::MultiplexGraph{T}) where {T <: Int} = T
eltype(multiplex)

Int64

### `edgetype`

In [10]:
edgetype(mp::MultiplexGraph{T}) where {T <: Int} = SimpleWeightedGraphs.edgetype(mp.retweet_graph)

edgetype(multiplex)

SimpleWeightedEdge{Int64,Int64}

### `inneighbors`

In [11]:
function inneighbors(mp::MultiplexGraph{T}, v::T, which::Symbol) where {T <: Int}
    return SimpleWeightedGraphs.inneighbors(getfield(mp, which) ,v)
end

inneighbors(multiplex,1, :follower_graph)

1-element Array{Int64,1}:
 3

### `ne`

In [12]:
function ne(mp::MultiplexGraph{T}, which::Symbol) where {T <: Int}
    return SimpleWeightedGraphs.ne(getfield(mp, which))
end

ne(multiplex, :favorite_graph)

1

### `nv`

In [13]:
function nv(mp::MultiplexGraph{T}, which::Symbol) where {T <: Int}
    return SimpleWeightedGraphs.nv(getfield(mp, which))
end

nv(multiplex, :retweet_graph)

3

### `outneighbors`

In [14]:
function outneighbors(mp::MultiplexGraph{T}, v::T, which::Symbol) where {T <: Int}
    return SimpleWeightedGraphs.outneighbors(getfield(mp, which),v)
end

outneighbors(multiplex, 2, :follower_graph)

0-element view(::Array{Int64,1}, 2:1) with eltype Int64

### `zero`

In [15]:
function zero(::MultiplexGraph{T})  where {T <: Int}
    return MultiplexGraph(0)
end

zero(multiplex)

MultiplexGraph{Int64}({0, 0} directed simple Int64 graph with Int64 weights, {0, 0} directed simple Int64 graph with Int64 weights, {0, 0} directed simple Int64 graph with Int64 weights)

### `is_directed`

In [16]:
function is_directed(::MultiplexGraph{T}) where {T <: Integer}
    return true
end

is_directed(multiplex)

true

### `get_edge`

In [17]:
function get_edge(mp::MultiplexGraph, s::Int, d::Int, which::Symbol)
    if has_edge(mp, s,d,which)
        return  SimpleWeightedGraphs.SimpleWeightedEdge(s,d, get_weight(mp,s,d,which))
    else
        return error("Error: specified source and destination in ",which," have no edge between them. Use `edges(mp::MultiplexGraph, which::String)` to get the list of all edges in subnetwork `which`")
    end
end

println(get_edge(multiplex, 3,1,:follower_graph))
get_edge(multiplex, 1,1,:follower_graph)

Edge 3 => 1 with weight 1


LoadError: Error: specified source and destination in follower_graph have no edge between them. Use `edges(mp::MultiplexGraph, which::String)` to get the list of all edges in subnetwork `which`

### `rem_edge!`

In [18]:
function rem_edge!(mp::MultiplexGraph, s::Int, d::Int, which::Symbol , weight::Int = 1)
    current_weight = get_weight(mp, s,d,which)
    if has_edge(mp,s,d,which) && current_weight >= weight
        if which == :follower_graph && !(weight in [1,0])
            return error("Error: follower graph weights must be 1 or 0.")
        else
            add_edge!(mp, s, d , which, -weight)
        end
    else
        return error("The required link does not exist, or the its weight is lesser than the weight to be subracted passed as argument.", "link weight = $current_weight , weight to be subtracted = $weight ")
    end
end

println(collect(edges(multiplex, :follower_graph)))
rem_edge!(multiplex, 1, 2, :follower_graph)
println(collect(edges(multiplex, :follower_graph)))
println(collect(edges(multiplex, :retweet_graph)))
rem_edge!(multiplex, 2, 3, :retweet_graph, 2)
println(collect(edges(multiplex, :retweet_graph)))
println(collect(edges(multiplex, :favorite_graph)))
rem_edge!(multiplex, 1, 3, :favorite_graph, 1)
println(collect(edges(multiplex, :favorite_graph)))
rem_edge!(multiplex, 1, 2, :follower_graph)

SimpleWeightedEdge{Int64,Int64}[Edge 1 => 2 with weight 1, Edge 3 => 1 with weight 1]
SimpleWeightedEdge{Int64,Int64}[Edge 3 => 1 with weight 1]
SimpleWeightedEdge{Int64,Int64}[Edge 2 => 3 with weight 3]
SimpleWeightedEdge{Int64,Int64}[Edge 2 => 3 with weight 1]
SimpleWeightedEdge{Int64,Int64}[Edge 1 => 3 with weight 2]
SimpleWeightedEdge{Int64,Int64}[Edge 1 => 3 with weight 1]


LoadError: The required link does not exist, or the its weight is lesser than the weight to be subracted passed as argument.link weight = 0 , weight to be subtracted = 1 

In [19]:
# swdg = SimpleWeightedDiGraph(2)
# SimpleWeightedGraphs.add_edge!(swdg,1,2,1)
# println(collect(SimpleWeightedGraphs.edges(swdg)))
# SimpleWeightedGraphs.add_edge!(swdg,1,2,2)
# println(collect(SimpleWeightedGraphs.edges(swdg)))